# Title <- Insert Title Here

*Jack Yao, Jessie Lu, Kexin Feng, Vincent Luong*

# Introduction

# Preliminary Results

In [23]:
library(tidyverse)
library(repr)
library(rvest)
library(readxl)
library(tidymodels)
library(infer)

### Reading Data from the Web

In [22]:
#Reading data  from UCI Machine Learning Repository
url <- "https://raw.githubusercontent.com/j-ackyao/stat-201-project/main/data/student-mat.csv?token=GHSAT0AAAAAACFRQAGNT7CEO35R3RGLCLUGZGAZUYA"
student_data <- read_delim(url, delim = ";") |>
    mutate(final_grade = G3) |>
    mutate(first_period_grade = G1) |>
    mutate(second_period_grade = G2) |>
    mutate(mother_education = as_factor (Medu)) |>
    mutate(father_education = as_factor (Fedu)) |>
    select(mother_education, father_education, first_period_grade, second_period_grade, final_grade)
    
head(student_data)

Rows: 395 Columns: 33
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (17): school, sex, address, famsize, Pstatus, Mjob, Fjob, reason, guardi...
dbl (16): age, Medu, Fedu, traveltime, studytime, failures, famrel, freetime...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


mother_education,father_education,first_period_grade,second_period_grade,final_grade
<fct>,<fct>,<dbl>,<dbl>,<dbl>
4,4,5,6,6
1,1,5,5,6
1,1,7,8,10
4,2,15,14,15
3,3,6,10,10
4,3,15,15,15


**Reading Data:** Reading and processing the Student Performance dataset from the web

# Methods: Plan

# References

https://archive.ics.uci.edu/dataset/320/student+performance <- cite this